# Run

This file will attempt to create a chain that can be given a stock and then use that to find financial and news, after which the results of that output can be used to recommend a buy or sell position.


```mermaid
flowchart TD
    A["Input: Stock"]
    B["Input: Strategy"]
    C["Agent"]
    D["News"]
    E["Financials"]
    F["Popular opinion"]
    H["LLM"]
    I["Output: Recommendation for the strategy"]
    A --> C
    C --> D
    C --> E
    C --> F
    D --> |Summarized| H
    E --> |Summarized| H
    F --> |Summarized| H
    B --> H
    A --> H
    H --> I
```


## Start


In [195]:
import dotenv
import os
from langchain_openai import OpenAI

dotenv.load_dotenv()


True

In [196]:
llm = OpenAI(temperature=0.7)
VERBOSE = False


## Add tools


In [197]:
from langchain.tools import BaseTool
from langchain_community.utilities import GoogleSearchAPIWrapper

search = GoogleSearchAPIWrapper()


class SearchTool(BaseTool):
    name = "Google Search"
    description = "Search Google for recent results. Input is a string."

    def _run(self, query: str):
        search = GoogleSearchAPIWrapper()
        return search.results(query=query, num_results=5)

    def _arun(self, query: str):
        raise NotImplementedError


tools = [SearchTool()]


In [198]:
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub


prompt = hub.pull("hwchase17/react")

searchAgent = create_react_agent(
    prompt=prompt,
    llm=llm,
    tools=tools,
)

search_executor = AgentExecutor(agent=searchAgent, tools=tools, verbose=VERBOSE)


## Creating LLMChains


In [199]:
from langchain_core.prompts import PromptTemplate


def search(company: str, search_executor: AgentExecutor) -> dict[str, str]:
    templates = {
        "positive_news": "Provide an overview of the most recent positive news that might have an impact on the stock price of {company}.",
        "negative_news": "Provide an overview of the most recent negative news that might have an impact on the stock price of {company}.",
        "financial": "Provide an overview of the most recent financial information about {company}.",
        "general_opinion": "Provide an overview of the general opinion investors have about {company}.",
    }

    prompts = {}
    for key, template in templates.items():
        prompts[key] = PromptTemplate.from_template(template)

    results = {}
    for key, prompt in prompts.items():
        results[key] = search_executor.invoke(
            input={"input": prompt.invoke(input={"company": company})}
        )["output"]

    return results


search_results = search("Nikola Corporation", search_executor)


In [200]:
search_results

{'positive_news': 'The recent positive news that might have an impact on the stock price of Nikola Corporation is that they have surpassed 200 orders for their hydrogen fuel cell trucks, their stockholder meeting proposal passed, and they are dedicated to advancing innovative zero-emissions truck solutions.',
 'negative_news': 'The most recent negative news that may have impacted the stock price of Nikola Corporation includes allegations of fraud, negative earnings reports, and a partnership with Iveco Group that resulted in a negative cash impact. The stock price of Nikola Corporation has also seen significant changes in the past few months, with a decrease of 22.44% year-to-date. It is important to analyze the correlation between negative news and stock price changes when considering investing in this company.',
 'financial': 'The most recent financial information about Nikola Corporation can be found in their fourth quarter and full year 2021 and 2022 reports and press releases.',
 

In [201]:
# Use this output to analyze the stock
from langchain.output_parsers import ResponseSchema, StructuredOutputParser


template = """
Given the following information about {company}, formulate a buy/sell/hold recommendation for the stock. Add your reasoning for the recommendation. 
You're allowed to formulate another strategy if you think it's more appropriate.

<<STRATEGY>>
{strategy}

<<INPUT>>
{information}

<<OUTPUT>>
{expected_output}
"""

recommendation_schema = ResponseSchema(
    name="Recommendation",
    description="Is the recommended move a buy, sell or a hold given the provided strategy?",
)
reasoning_schema = ResponseSchema(
    name="Reasoning",
    description="Reasoning behind the recommendation and the strategy used.",
)

alternate_strategy_schema = ResponseSchema(
    name="Alternate Strategy",
    description="If you think the recommended strategy is not appropriate, provide an alternate strategy.",
)


output_parser = StructuredOutputParser.from_response_schemas(
    [recommendation_schema, reasoning_schema, alternate_strategy_schema]
)

format_instructions = output_parser.get_format_instructions()

prompt = PromptTemplate(
    template=template,
    input_variables=["company", "strategy", "information"],
    partial_variables={"expected_output": format_instructions},
    input_types={
        "company": "string",
        "strategy": "string",
        "information": "dict[string, string]",
        "expected_output": "string",
    },
)

format_instructions


'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"Recommendation": string  // Is the recommended move a buy, sell or a hold given the provided strategy?\n\t"Reasoning": string  // Reasoning behind the recommendation and the strategy used.\n\t"Alternate Strategy": string  // If you think the recommended strategy is not appropriate, provide an alternate strategy.\n}\n```'

In [202]:
# Create an LLMChain

from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt, output_parser=output_parser)

chain.invoke(
    input={
        "company": "Nikola Corporation",
        "strategy": "Buy the stock now and keep it until I have 5% profit, at which point I'll sell my stake.",
        "information": search_results,
    }
)


{'company': 'Nikola Corporation',
 'strategy': "Buy the stock now and keep it until I have 5% profit, at which point I'll sell my stake.",
 'information': {'positive_news': 'The recent positive news that might have an impact on the stock price of Nikola Corporation is that they have surpassed 200 orders for their hydrogen fuel cell trucks, their stockholder meeting proposal passed, and they are dedicated to advancing innovative zero-emissions truck solutions.',
  'negative_news': 'The most recent negative news that may have impacted the stock price of Nikola Corporation includes allegations of fraud, negative earnings reports, and a partnership with Iveco Group that resulted in a negative cash impact. The stock price of Nikola Corporation has also seen significant changes in the past few months, with a decrease of 22.44% year-to-date. It is important to analyze the correlation between negative news and stock price changes when considering investing in this company.',
  'financial': 'Th